In [10]:
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm
import threading
import time as t_time
from ratelimit import limits

In [11]:
!pip install ratelimit

### Lectura de la data

In [12]:
ticker_library = pd.read_csv('tickers.csv')

ticker_selected = pd.read_csv('SP500_component_stocks.csv',header = None)
ticker_selected.columns = ['name','ticker']

display(ticker_selected)


/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,ticker
0,Agilent Technologies Inc.,A
1,American Airlines Group Inc.,AAL
2,Advance Auto Parts Inc.,AAP
3,Apple Inc.,AAPL
4,AbbVie Inc.,ABBV
...,...,...
500,Xylem Inc.,XYL
501,Yum! Brands Inc.,YUM
502,Zimmer Biomet Holdings Inc.,ZBH
503,Zions Bancorporation N.A.,ZION


### Preparación de cik&ticker dataframe

In [13]:
ticker_cik_df = pd.DataFrame()

ticker_list = ticker_selected.ticker.unique()

cik_list = []

for ticker in ticker_list:    
    try:
        cik_list.append(list(ticker_library[ticker_library.ticker == ticker].secfilings)[0][-10:])
        
    except:
        cik_list.append('')

ticker_cik_df['cik'] = cik_list
ticker_cik_df['ticker'] = ticker_list

ticker_cik_df = ticker_cik_df[ticker_cik_df['cik'] != '']

ticker_cik_df

,cik,ticker
0,0001090872,A
1,0000006201,AAL
2,0001158449,AAP
3,0000320193,AAPL
4,0001551152,ABBV
...,...,...
500,0001524472,XYL
501,0001041061,YUM
502,0001136869,ZBH
503,0000109380,ZION


In [14]:
listtickers = ['AMZN','BBY','BKNG','MCD','EBAY','F','HD','TGT','WHR','JPM','SIVB','CFG','C','ALL','IVZ','ETFC','MET','PFG','CBOE',
              'CTL','IPG','VIAC','NFLX','CHTR','FB','TWTR','NWSA','FOXA','AMD','INTC','AAPL','LRCX','MSFT','NLOK','CTSH','ADS',
              'WU','PAYC','ABT','CVS','PFE','JNJ','BIIB','INCY','HSIC','WAT','ALGN','EW']
listrows = []
for j in listtickers:
    row = ticker_cik_df[ticker_cik_df['ticker'] == j]
    listrows.append(row)

In [15]:
ticker_cik_sample = pd.DataFrame()
for i in range(len(listrows)):
    ticker_cik_sample = pd.concat([ticker_cik_sample,listrows[i]],ignore_index=False)
ticker_cik_sample

,cik,ticker
39,0001018724,AMZN
64,0000764478,BBY
71,0001075531,BKNG
304,0000063908,MCD
156,0001065088,EBAY
178,0000037996,F
222,0000354950,HD
435,0000027419,TGT
485,0000106640,WHR
266,0000019617,JPM


In [16]:
ticker_cik_sample = ticker_cik_df.sample(n=2,replace=False,random_state=0)
ticker_cik_sample

,cik,ticker
92,0000816284,CELG
257,0000049826,ITW


In [17]:
ticker_cik_sample

,cik,ticker
92,0000816284,CELG
257,0000049826,ITW


### import function to scrape 

In [18]:
def WriteLogFile(log_file_name, text):
    with open(log_file_name, "a") as log_file:
        log_file.write(text)

    return

In [19]:
def Scrape10K(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    try:
        os.mkdir(cik)
    except OSError:
        print("Already scraped CIK", cik)
        return
    
    os.chdir(cik)
    
    res = requests.get(browse_url_base % cik)
    
    if res.status_code != 200:
        os.chdir('..')
        os.rmdir(cik) # remove empty dir
        text = "Request failed with error code " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        WriteLogFile(log_file_name, text)
        return

    
    soup = bs.BeautifulSoup(res.text, "lxml")

    html_tables = soup.find_all('table')
    
    if len(html_tables)<3:
        os.chdir('..')
        return
    
    filings_table = pd.read_html(str(html_tables[2]), header=0)[0]
    filings_table['Filings'] = [str(x) for x in filings_table['Filings']]

    filings_table = filings_table[(filings_table['Filings'] == '10-K') | (filings_table['Filings'] == '10-K405')]

    if len(filings_table)==0:
        os.chdir('..')
        return
    
    filings_table['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in filings_table['Description']]

    for index, row in filings_table.iterrows():
        
        acc_no = str(row['Acc_No'])
        
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        
        if docs_page.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue

        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        docs_table = docs_table[(docs_table['Type'] == '10-K') | (docs_table['Type'] == '10-K405')]
        
        if len(docs_table)==0:
            continue
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
        
        if str(docname) == 'nan':
            os.chdir('..')
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue       
        
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname))
        
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
        
        """
        date = str(row['Filing Date'])
        filename = cik + '_' + date + '.txt'
        html_file = open(filename, 'a')
        html_file.write(file.text)
        html_file.close()
        """
        if '.txt' in docname:
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        
    os.chdir('..')
        
    return

In [20]:
def Scrape10Q(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    try:
        os.mkdir(cik)
    except OSError:
        print("Already scraped CIK", cik)
        return
    
    os.chdir(cik)
    
    """
    print('Scraping CIK', cik)
    """
    
    res = requests.get(browse_url_base % cik)
    
    if res.status_code != 200:
        os.chdir('..')
        os.rmdir(cik) # remove empty dir
        text = "Request failed with error code " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        WriteLogFile(log_file_name, text)
        return
    

    soup = bs.BeautifulSoup(res.text, "lxml")

    html_tables = soup.find_all('table')
    
    if len(html_tables)<3:
        print("table too short")
        os.chdir('..')
        return
    
    filings_table = pd.read_html(str(html_tables[2]), header=0)[0]
    filings_table['Filings'] = [str(x) for x in filings_table['Filings']]

    filings_table = filings_table[filings_table['Filings'] == '10-Q']

    if len(filings_table)==0:
        os.chdir('..')
        return
    
    filings_table['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in filings_table['Description']]

    for index, row in filings_table.iterrows():
        
        acc_no = str(row['Acc_No'])
        
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        
        if docs_page.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
            
        
        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        docs_table = docs_table[docs_table['Type'] == '10-Q']
        
        if len(docs_table)==0:
            continue
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
        
        if str(docname) == 'nan':
            os.chdir('..')
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue       
        
        
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname))
        
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
            
        
        """
        date = str(row['Filing Date'])
        filename = cik + '_' + date + '.txt'
        html_file = open(filename, 'a')
        html_file.write(file.text)
        html_file.close()
        """
        if '.txt' in docname:
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        
    os.chdir('..')
        
    return

### Definir Direcciones

In [21]:
pathname_10k = './Scrape_data/10_k'
pathname_10q = './Scrape_data/10_q'

### Buscar Data

In [22]:

browse_url_base_10k = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-K'
filing_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

try:
    os.chdir(pathname_10k)
except Exception:
    os.makedirs(pathname_10k)
    os.chdir(pathname_10k)

time = strftime("%Y-%m-%d %Hh%Mm%Ss", gmtime())
log_file_name = 'log '+time+'.txt'
with open(log_file_name, 'a') as log_file:
    log_file.close()

for cik in tqdm(ticker_cik_sample['cik']):
    Scrape10K(browse_url_base=browse_url_base_10k, 
          filing_url_base=filing_url_base_10k, 
          doc_url_base=doc_url_base_10k, 
          cik=cik,
          log_file_name=log_file_name)
    

os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [00:27<00:00, 13.63s/it]


In [23]:

browse_url_base_10q = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-Q&count=1000'
filing_url_base_10q = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10q = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

try:
    os.chdir(pathname_10q)
except Exception:
    os.makedirs(pathname_10q)
    os.chdir(pathname_10q)

time = strftime("%Y-%m-%d %Hh%Mm%Ss", gmtime())
log_file_name = 'log '+time+'.txt'
log_file = open(log_file_name, 'a')
log_file.close()

for cik in tqdm(ticker_cik_sample['cik']):
    Scrape10Q(browse_url_base = browse_url_base_10q, 
          filing_url_base = filing_url_base_10q, 
          doc_url_base = doc_url_base_10q, 
          cik = cik,
          log_file_name = log_file_name)

os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [01:19<00:00, 39.97s/it]


### Conversión de HTML a txt limpio

In [24]:
def RemoveNumericalTables(soup):
    
    def GetDigitPercentage(tablestring):
        if len(tablestring)>0.0:
            numbers = sum([char.isdigit() for char in tablestring])
            length = len(tablestring)
            return numbers/length
        else:
            return 1
    
    [x.extract() for x in soup.find_all('table') if GetDigitPercentage(x.get_text())>0.15]
    
    return soup

In [25]:
def RemoveTags(soup):
    
    text = soup.get_text()
    
    text = text.replace('\n', ' ')
    
    '''
    Adding a replace to convert all excecute space to one space 
    '''
    text = re.sub(r'\s', ' ', text)
    
    text = unicodedata.normalize('NFKD', text)
    
    return text

In [26]:
def ConvertHTML(cik):
    
    try: 
        os.chdir(cik)
    except FileNotFoundError:
        print("Could not find directory for CIK", cik)
        return
    """ avoid print"""    
    
    parsed = False # flag to tell if we've parsed anything
    
    try:
        os.mkdir('rawtext')
    except OSError:
        pass
    
    file_list = [fname for fname in os.listdir() if not (fname.startswith('.') | os.path.isdir(fname))]
    
    for filename in file_list:
            
        new_filename = filename.replace('.html', '.txt')
        text_file_list = os.listdir('rawtext')
        if new_filename in text_file_list:
            continue
        
        
        with open(filename, 'r',encoding='utf-8') as file:
            parsed = True
            soup = bs.BeautifulSoup(file.read(), "lxml")
            soup = RemoveNumericalTables(soup)
            
            text = RemoveTags(soup)
            with open('rawtext/'+new_filename, 'w',encoding='utf-8') as newfile:
                newfile.write(text)
    
    if parsed==False:
        print("Already parsed CIK", cik)
    
    os.chdir('..')
    return

In [27]:
def delete_reference(soup):
    
    management_str = "Financial Condition and Results of Operations"
                    
    financial_str = "and Supplementary Data"
    
    length_1 = len('Management’s Discussion and Analysis of Financial Condition and Results of Operations')
    
    length_2 = len('Financial Statements and Supplementary Data')
    
    for part in soup.find_all(text = re.compile(management_str)):
        
        if len(part) > length_1:
            
            part.extract()
            
    for part in soup.find_all(text = re.compile(financial_str)):
        if len(part) > length_2:
            part.extract()
    
    return soup


### Aplicación de la conversión HTML

In [28]:
os.chdir(pathname_10k)

for cik in tqdm(ticker_cik_sample['cik']):
    ConvertHTML(cik)
os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [00:29<00:00, 14.88s/it]


In [29]:

os.chdir(pathname_10q)

for cik in tqdm(ticker_cik_sample['cik']):
    ConvertHTML(cik)
os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [00:50<00:00, 25.08s/it]


In [30]:
os.getcwd()

'/Universidad/TESIS'

In [31]:
os.chdir('..')